### Imports

In [1]:
from random import random
from math   import log, e, gamma, factorial, pi, cos, sin, atan, tan
from tabulate import tabulate
import time
import numpy as np

### Funciones generales

In [2]:
def estimar_esperanza(N, generador):
    """
    Estima la esperanza usando la ley fuerte de los grandes numeros
    Args:
        N:            Cantidad total de valores a promediar
        generador:    Generador de la variable aleatoria de interes
    """
    esperanza = 0
    for _ in range(N):
        esperanza += generador()
    return esperanza / N

def estimar_varianza(N, generador, media):
    """
    Estima la varianza de una variable aleatoria
    Args:
        N:         Tamaño de la muestra
        generador: Generador de la variable aleatoria de interes 
        media:     Esperanza de la variable aleatoria
    """
    varianza = 0
    for _ in range(N):
        varianza += (generador() - media) ** 2
    varianza = varianza / N
    return varianza

def tiempo_ejecucion(funcion):
    """
    Retorna el tiempo en ms que demora funcion hasta terminar su ejecucion
    y el resultado de funcion
    Return:
        (Tiempo de ejecucion total, resultado de la funcion ejecutada)
    """
    tiempo = time.time()
    resultado_funcion = funcion()
    tiempo = (time.time() - tiempo) * 10**3
    return (np.round(tiempo,4), resultado_funcion)

### Ejercicio 1

In [3]:
def generar_1_a():
    """
    Genera la variable aleatoria continua dada en el ejercicio 1_a
    usando el metodo de la transformada inversa
    """
    U = random()
    if U <= 0.25:
        return 2 * (1 + U**(1/2))
    else:
        return 6 * (1 - (1 - (2+U) / 3)**(1/2))

In [4]:
def generar_1_b():
    """
    Genera la variable aleatoria continua dada en el ejercicio 1_b
    usando el metodo de la transformada inversa
    """
    U = random()
    if U <= 0.6:
        return (-18 + (18**2 + 420*U) ** (1/2)) / 6
    else:
        return ((U * 35 - 19)/2) ** (1/3)

In [5]:
def generar_1_c():
    """
    Genera la variable aleatoria continua dada en el ejercicio 1_c
    usando el metodo de la transformada inversa
    """
    U = random()
    if U <= 1/16:
        return log(16*U) / 4
    else:
        return (U - 1/16) * 4

### Ejercicio 2

In [6]:
def generar_pareto(a):
    """
    Genera una variable aleatoria pareto usando el metodo de la transformada
    inversa con escala xm = 1
    Args:
        a: forma de la variable de pareto
    """
    U = random()
    return (1 / (1 - U)) ** (1/a)

In [7]:
def generar_exp(Lambda):
    """
    Genera una variable aleatoria exponencial usando el metodo de la transformada
    inversa
    Args:
        Lambda: Parametro de la variable aleatoria exponencial
    """
    U = random()
    return -log(U) / Lambda

def generar_gamma(n, Lambda):
    """
    Genera una variable aleatoria Gamma como suma de variables exponenciales
    Args:
        n, Lambda: Parametros de la variable aleatoria Gamma
    """
    U = 1
    for _ in range(n):
        U *= 1 - random()
    
    return - log(U) / Lambda
    
def generar_erlang(k, mu):
    """
    Genera una variable aleatoria de erlang usando un generador de variables
    Gamma
    """
    return generar_gamma(k, mu)

In [8]:
def generar_weibull(beta, Lambda):
    """
    Genera una variable aleatoria weibull usando el metodo de transformada inversa
    X ~ Weibull(beta, Lambda)
    """
    U = random()
    U = 1 / (1 - U)
    return log(U) ** (1/beta) / Lambda

In [9]:
a, mu, k, beta, Lambda = 2, 2, 2, 2, 1
N = 10**4
# Esperanzas estimadas
E_pareto_estimada  = estimar_esperanza(N, lambda: generar_pareto(a))
E_erlang_estimada  = estimar_esperanza(N, lambda: generar_erlang(k, mu)) 
E_weibull_estimada = estimar_esperanza(N, lambda: generar_weibull(beta, Lambda))

# Esperanzas reales
E_pareto_real  = a / (a - 1)
E_erlang_real  = k / mu
E_weibull_real = gamma(1 + 1/beta) / Lambda

datos_2 = [["i"  , E_pareto_real , E_pareto_estimada],
           ["ii" , E_erlang_real , E_erlang_estimada],
           ["iii", E_weibull_real, E_weibull_estimada]
          ]
header_2 = ["E[X]", "valor real", "valor estimado"]
print(tabulate(datos_2, headers=header_2, tablefmt="pretty"))

+------+-------------------+--------------------+
| E[X] |    valor real     |   valor estimado   |
+------+-------------------+--------------------+
|  i   |        2.0        | 1.9744102815520665 |
|  ii  |        1.0        | 1.002870899174326  |
| iii  | 0.886226925452758 | 0.892357926796895  |
+------+-------------------+--------------------+


### Ejercicio 3

In [10]:
# Genere datos usando tres exponenciales independientes con media 3,5 y 7 
# respectivamente y p = (0.5, 0.3, 0.2)

def variable_ej3():
    """
    Genera la variable aleatoria del ejercicio 3 usando el metodo de composicion
    """
    p       = [0.5, 0.3, 0.2] # Ponderacion de la combinacion lineal
    lambdas = [1/3, 1/5, 1/7] # Parametro de las variables exponenciales
    U, i, S = random(), 0, p[0]
    while U > S:
        i += 1
        S += p[i]
    
    return generar_exp(lambdas[i])

In [11]:
N = 10**5
esperanza_estimada_3 = estimar_esperanza(N, variable_ej3)
esperanza_real_3     = 3*0.5 + 5*0.3 + 0.2*7  # Usando linealidad de la esperanza
print("""
Esperanza exacta:   {}
Esperanza estimada: {}
""".format(esperanza_real_3, esperanza_estimada_3))


Esperanza exacta:   4.4
Esperanza estimada: 4.418536576048766



### Ejercicio 4

In [12]:
def generar_4():
    """
    Genera la variable aleatoria descrita en el ejercicio 4
    """
    Y = generar_exp(1)
    U = random()
    return U ** (1 / Y)

### Ejercicio 5

In [13]:
def generar_M(generadores):
    """
    Genera la variable aleatoria M descrita en el ejercicio 5
    Args:
        generadores: Lista de generadores de las variables aleatorias involucradas
    """
    M = generadores[0]()
    for gen in generadores[1:]:
        M = max(M, gen())
    return M

def generar_m(generadores):
    """
    Genera la variable aleatoria m descrita en el ejercicio 5
    Args:
        generadores: Lista de generadores de la variables aleatorias involucradas
    """
    m = generadores[0]()
    for gen in generadores[1:]:
        m = min(m, gen())
    return m

In [14]:
# Genere una muestra de 10 valores de las variables M y m con distribuciones 
# FM y Fm si Xi son exponenciales independientes con parámetros 1,2 y 3 respectivamente

# Esperanza exacta de m
# Sabemos que por resultados teoricos esa variable es exponencial de parametro 1*2*3
esperanza_m = 1 / (1 * 2 * 3)

# Esperanza exacta de M
# Por un resultado teorico de internet tenemos que la esperanza es igual a 
esperanza_M = 1/1 + 1/2 + 1/3 - 1/(1+2) - 1/(1+3) - 1/(2+3) + 1/(1+2+3) 

# Estimamos las esperanzas
generadores = [lambda: generar_exp(1), lambda: generar_exp(2), lambda: generar_exp(3)]
N = 10**5
esperanza_estimada_m = estimar_esperanza(N, lambda: generar_m(generadores))
esperanza_estimada_M = estimar_esperanza(N, lambda: generar_M(generadores))


# Hacemos una tabla
data_5 = [["M", esperanza_M, esperanza_estimada_M],
          ["m", esperanza_m, esperanza_estimada_m]
         ]
headers_5 = ["Variable", "Esperanza exacta", "Esperanza estimada"]

print(tabulate(data_5, headers=headers_5, tablefmt="pretty"))

+----------+---------------------+---------------------+
| Variable |  Esperanza exacta   | Esperanza estimada  |
+----------+---------------------+---------------------+
|    M     | 1.2166666666666668  | 1.2145962992721628  |
|    m     | 0.16666666666666666 | 0.16722946863355637 |
+----------+---------------------+---------------------+


### Ejercicio 6

In [15]:
def generar_6_TI(n):
    """
    Genera la variable aleatoria del ejercicio 6 usando el metodo de la 
    transformada inversa
    """
    U = random()
    return U ** (1/n)

def generar_6_M(n):
    """
    Genera la variable aleatoria del ejercicio 6 usando el resultado del
    ejercicio 5, la variable aleatoria M
    """
    return generar_M([random]*n)

def generar_6_AYR(n):
    """
    Genera la variable aleatoria del ejercicio 6 usando el metodo de aceptacion 
    y rechazo a partir de una v.a. Y ~ U(0, 1)
    """
    f_y = 1
    f_x = lambda x: n * x**(n-1)
    c   = n
    while(True):
        Y = random()
        U = random()
        if U < f_x(Y) / (n * f_y):
            return Y

In [16]:
# Comparamos los tiempos de ejecucion
# la esperanza real de esta variable aleatoria es n / (n+1)

n = 10
N = 10**5

esperanza_6_real = n
tiempo_TI  , esperanza_TI  = tiempo_ejecucion(lambda: estimar_esperanza(N, lambda: generar_6_TI(n)))
tiempo_M   , esperanza_M   = tiempo_ejecucion(lambda: estimar_esperanza(N, lambda: generar_6_M(n)))
tiempo_AYR , esperanza_AYR = tiempo_ejecucion(lambda: estimar_esperanza(N, lambda: generar_6_AYR(n)))


datos_6 = [["TI" , esperanza_TI , tiempo_TI ],
           ["M"  , esperanza_M  , tiempo_M  ],
           ["AYR", esperanza_AYR, tiempo_AYR]
          ]
headers_6 = ["Algoritmo", "Esperanza estimada", "Tiempo requerido (ms)"]

print(tabulate(datos_6, headers=headers_6, tablefmt="pretty"))
print("Esperanza real", n / (n+1))

+-----------+--------------------+-----------------------+
| Algoritmo | Esperanza estimada | Tiempo requerido (ms) |
+-----------+--------------------+-----------------------+
|    TI     | 0.9091640853564626 |        18.5199        |
|     M     | 0.9092202561752841 |       143.1956        |
|    AYR    | 0.9091854350230144 |       232.0123        |
+-----------+--------------------+-----------------------+
Esperanza real 0.9090909090909091


### Ejercicio 7

In [17]:
def generar_7_TI():
    """
    Genera la variable aleatoria descrita en el ejercicio 7 usando el
    metodo de la transformada inversa
    """
    U = random()
    return e ** U

def generar_7_AYR():
    """
    Genera la variable aleatoria descrita en el ejercicio 7 usando el
    metodo de aceptacion y rechazo a partir una variable Y ~ U(0, e)
    """
    f = lambda x: 1/x * (x >= 1 and x <= e)    # Pues f solo es no nula en ese intervalo
    # g_y = 1/e por ser uniforme continua
    # c   = e por analisis previo
    # g_y * c = 1
    while True:
        Y = random() * e
        U = random()
        if U < f(Y):
            return Y

In [18]:
# Esperanza real
esperanza_7_real = e - 1

# Estimamos las esperanzas
N = 10**5
tiempo_7_TI , esperanza_7_TI  = tiempo_ejecucion(lambda: estimar_esperanza(N, lambda: generar_7_TI()))
tiempo_7_AYR, esperanza_7_AYR = tiempo_ejecucion(lambda: estimar_esperanza(N, lambda: generar_7_AYR()))

# P(X <= 2) = log(2)
px_real = log(2)
px_TI   = 0
px_AYR  = 0
for _ in range(N):
    if generar_7_TI() <= 2: 
        px_TI += 1
    
    if generar_7_AYR() <= 2:
        px_AYR += 1

px_TI  = px_TI  / N
px_AYR = px_AYR / N 


datos_7 = [["TI" , esperanza_7_TI , tiempo_7_TI , px_TI ],
           ["AYR", esperanza_7_AYR, tiempo_7_AYR, px_AYR]
          ]

headers_7 = ["Algoritmo", "Esperanza estimada", "Tiempo esperanza (ms)", "P[X<=2]"]
print(tabulate(datos_7, headers=headers_7, tablefmt="pretty"))
print("Esperanza real: ", esperanza_7_real)
print("P[X<=2] real:   "  , px_real)

+-----------+--------------------+-----------------------+---------+
| Algoritmo | Esperanza estimada | Tiempo esperanza (ms) | P[X<=2] |
+-----------+--------------------+-----------------------+---------+
|    TI     | 1.7183867696060757 |        22.2254        | 0.69499 |
|    AYR    | 1.7176038749085918 |        70.9484        | 0.69624 |
+-----------+--------------------+-----------------------+---------+
Esperanza real:  1.718281828459045
P[X<=2] real:    0.6931471805599453


### Ejercicio 8

In [19]:
def generar_8():
    """
    Genera la variable aleatoria del ejercicio 8 como suma de dos variables
    uniformes
    """
    U, V = random(), random()
    return U + V

def generar_8_TI():
    """
    Genera la variable aleatoria del ejercicio 8 usando el metodo de la 
    transformada inversa
    """
    U = random()
    if U <= 0.5:
        return (U*2) ** (1/2)
    else: 
        return 2 - (2 - 2*U)**(1/2)
def generar_8_AYR():
    """
    Genera la variable aleatoria del ejercicio 8 usando el metodo de 
    aceptacion y rechazo con una variable Y ~ U(0, 2)
    """
    f = lambda x: x * (x>=0 and x<=1) + (2-x) * (x>=1 and x<=2)
    # c   = 2
    # g_y = 1/2
    # c * g_y = 1
    
    while True:
        Y = random() * 2
        U = random()
        if U < f(Y):
            return Y

In [20]:
N = 10**5

# Estimacion de la esperanza y eficiencia

tiempo_8, e_8         = tiempo_ejecucion(lambda: estimar_esperanza(N, lambda: generar_8()))
tiempo_8_TI, e_8_TI   = tiempo_ejecucion(lambda: estimar_esperanza(N, lambda: generar_8_TI()))
tiempo_8_AYR, e_8_AYR = tiempo_ejecucion(lambda: estimar_esperanza(N, lambda: generar_8_AYR()))


# P(X > x0) donde x0 = 1.5
px_real = 0.125
px_SUM  = 0
px_TI   = 0
px_AYR  = 0
for _ in range(N):
    if generar_8() > 1.5:
        px_SUM += 1
    
    if generar_8_TI() > 1.5: 
        px_TI += 1
    
    if generar_8_AYR() > 1.5:
        px_AYR += 1

px_SUM = px_SUM / N
px_TI  = px_TI  / N
px_AYR = px_AYR / N 

data_8 = [["Suma", e_8    , tiempo_8    , px_SUM ],
          ["TI"  , e_8_TI , tiempo_8_TI , px_TI  ], 
          ["AYR" , e_8_AYR, tiempo_8_AYR, px_AYR ]]

headers_8 = ["Algoritmo", "Esperanza", "Tiempo esperanza (ms)", "P(X>x0)"]

print(tabulate(data_8, headers=headers_8, tablefmt="pretty"))

+-----------+--------------------+-----------------------+---------+
| Algoritmo |     Esperanza      | Tiempo esperanza (ms) | P(X>x0) |
+-----------+--------------------+-----------------------+---------+
|   Suma    | 0.9994955784680849 |        21.946         | 0.12536 |
|    TI     | 1.0020463432967897 |        22.3284        | 0.12327 |
|    AYR    | 1.0025624926276981 |        70.8566        | 0.12433 |
+-----------+--------------------+-----------------------+---------+


### Ejercicio 9

In [21]:
def normal_ross(mu=0, sigma=1):
    """
    Genera una variable normal estandar (por defecto) usando el algoritmo descrito 
    en el libro de Sheldon Ross, el cual esta basado en el metodo de aceptacion y 
    rechazo
    Args:
        mu   : Esperanza de la variable normal
        sigma: Desvio estandar de la variable normal 
    """
    while True:
        Y1 = -log(random()) # Exponencial de parametro 1
        Y2 = -log(random()) # Exponencial de parametro 1
        if Y2 >= (Y1 - 1) ** 2 / 2:
            # En este punto Y1 es una variable aleatoria con distribucion |Z|
            # La siguiente comparacion es el metodo para generar Z con composicion
            # a partir del generador de |Z|
            if random() < 0.5:
                return Y1 * sigma + mu
            else:
                return -Y1 * sigma + mu

            
def normal_polar(mu=0, sigma=1):
    """
    Genera dos variables normales estandar (por defecto) usando el metodo polar
    Args:
        mu   : Esperanza de la variable normal
        sigma: Desvio estandar de la variable normal 
    Return:
        X, Y : Variables normales de parametros mu y sigma
    """
    r_cuadrado = -2 * log(1 - random())
    theta = 2 * pi * random()
    X = r_cuadrado**(1/2) * cos(theta)
    Y = r_cuadrado**(1/2) * sin(theta)
    return (X * sigma + mu, Y * sigma + mu) 


NV_MAGICCONST = 4 * e**(-0.5) / (2.0)**(1/2)
def normal_razon_uniformes(mu=0, sigma=1):
    """
    Genera dos variables normales estandar (por defecto) usando el metodo de 
    razon entre uniformes
    Args:
        mu   : Esperanza de la variable normal
        sigma: Desvio estandar de la variable normal 
    """
    while True:
        u1 = random()
        u2 = 1 - random()
        z  = NV_MAGICCONST * (u1 - 0.5) / u2
        zz = z * z / 4.0
        if zz <= -log(u2):
            break
    return mu + z * sigma

In [22]:
# Estimo la esperanza y comparo tiempos
N = 10**4
t_e_normal_ross , e_normal_ross  = tiempo_ejecucion(
    lambda: estimar_esperanza(N, lambda: normal_ross()))
t_e_normal_polar, e_normal_polar = tiempo_ejecucion(
    lambda: estimar_esperanza(int(N/2), lambda: sum(normal_polar())))
t_e_normal_razon, e_normal_razon = tiempo_ejecucion(
    lambda: estimar_esperanza(N, lambda: normal_razon_uniformes()))

# Estimo la varianza y comparo tiempos
t_v_normal_ross, v_normal_ross = tiempo_ejecucion(
    lambda: estimar_varianza(N, lambda: normal_ross(), media=0))
t_v_normal_polar, v_normal_polar = tiempo_ejecucion(
    lambda: estimar_varianza(int(N/2), lambda: sum(normal_polar()), media=0))
t_v_normal_razon, v_normal_razon = tiempo_ejecucion(
    lambda: estimar_varianza(N, lambda: normal_razon_uniformes(), media=0))

# Ajusto las estimaciones polares porque generamos N/2 pares y los algoritmos
# promedian con ese N/2 por lo cual debemos divir por 2 al resultado

e_normal_polar = e_normal_polar / 2
v_normal_polar = v_normal_polar / 2
                              
# Tabla comparativa
data_9 = [
    ["Ross" , e_normal_ross , t_e_normal_ross , v_normal_ross , t_v_normal_ross ],
    ["Polar", e_normal_polar, t_e_normal_polar, v_normal_polar, t_v_normal_polar],
    ["Razon", e_normal_razon, t_e_normal_razon, v_normal_razon, t_v_normal_razon],
]

headers_9 = ["Algoritmo", "Esperanza", "Tiempo(ms)", "Varianza", "Tiempo(ms)"]

print(tabulate(data_9, headers=headers_9, tablefmt="pretty"))

+-----------+-----------------------+------------+--------------------+------------+
| Algoritmo |       Esperanza       | Tiempo(ms) |      Varianza      | Tiempo(ms) |
+-----------+-----------------------+------------+--------------------+------------+
|   Ross    | 0.002460412253742295  |   8.3768   | 1.0065115079420288 |   7.7386   |
|   Polar   | 0.0054213614132910945 |   3.3927   | 0.9995351056622851 |   3.8972   |
|   Razon   |  0.01372181370690598  |   4.5788   | 0.9876628315867166 |   5.4791   |
+-----------+-----------------------+------------+--------------------+------------+


### Ejercicio 11

In [23]:
radius = (1/pi) ** (1/2)
def generar_cauchy(Lambda=1):
    """
    Genera una variable aleatoria de Cauchy de parametro Lambda (por defecto 1)
    usando el metodo de razon entre uniformes
    """
    while True:
        U = random() * radius
        V = ((random() * 2) - 1) * radius
        if U**2 + V**2 < radius**2:
            return V/U * Lambda

def acumulada_cauchy(x, Lambda=1):
    """
    Calcula la probabilidad acumulada de una variable de cauchy de parametro Lambda
    (1 por defecto) 
    return:
        F(x)
    """
    return (2*atan(x/Lambda) + pi) / (2 * pi) 

def prob_intervalo(alpha, generador, N):
    """
    Estima la probabilidad de que la variable aleatoria generada por generador
    este en el intervalo [-alpha, alpha]
    Args:
        alpha:     limite del intervalo
        generador: generador de variable aleatoria
        N:         cantidad de valores a generar
    """
    en_intervalo = 0
    for _ in range(N):
        X = generador()
        if abs(X) <= abs(alpha):
            en_intervalo += 1
    return en_intervalo / N

In [24]:
# Si X es una variable aleatoria continua de cauchy de parametro 1
# P(-alpha <= X <= alpha) = F(alpha) - F(-alpha)
p_lambda = lambda x: acumulada_cauchy(x) - acumulada_cauchy(-x)

Lambdas  = [1, 2.5, 0.3]
p_reales = list(map(p_lambda, Lambdas))

# Estimo las probabilidades anteriores con 10000 valores
N = 10000
p_estimadas = list(map(lambda x: prob_intervalo(x, generar_cauchy, N), Lambdas))

data_11    = np.array([Lambdas, p_reales, p_estimadas]).T
headers_11 = ["Lambda (l)", "P(-l<=x<=l) real", "P(-l<=x<=l) estimado"]

print(tabulate(data_11, headers=headers_11, tablefmt="pretty"))

+------------+---------------------+----------------------+
| Lambda (l) |  P(-l<=x<=l) real   | P(-l<=x<=l) estimado |
+------------+---------------------+----------------------+
|    1.0     |         0.5         |        0.4974        |
|    2.5     | 0.7577621168183133  |        0.7505        |
|    0.3     | 0.18554715815548478 |        0.1877        |
+------------+---------------------+----------------------+


### Ejercicio 12

In [25]:
def generar_cauchy_TI(Lambda=1):
    """
    Genera una variable aleatoria cauchy de parametro Lambda (1 por defecto) usando
    el metodo de la transformada inversa
    """
    U = random()
    return tan((2*pi*U - pi)/2) * Lambda

In [26]:
# Si X es una variable aleatoria continua de cauchy de parametro 1
# P(-alpha <= X <= alpha) = F(alpha) - F(-alpha)
p_lambda = lambda x: acumulada_cauchy(x) - acumulada_cauchy(-x)

Lambdas  = [1, 2.5, 0.3]
p_reales = list(map(p_lambda, Lambdas))

# Estimo las probabilidades anteriores con 10000 valores
N = 10000
p_estimadas = list(map(lambda x: prob_intervalo(x, generar_cauchy_TI, N), Lambdas))

data_11    = np.array([Lambdas, p_reales, p_estimadas]).T
headers_11 = ["Lambda (l)", "P(-l<=x<=l) real", "P(-l<=x<=l) estimado"]

print(tabulate(data_11, headers=headers_11, tablefmt="pretty"))

+------------+---------------------+----------------------+
| Lambda (l) |  P(-l<=x<=l) real   | P(-l<=x<=l) estimado |
+------------+---------------------+----------------------+
|    1.0     |         0.5         |        0.493         |
|    2.5     | 0.7577621168183133  |        0.7626        |
|    0.3     | 0.18554715815548478 |        0.1872        |
+------------+---------------------+----------------------+


In [27]:
# Comparacion de eficiencia. 
# Generamos N = 10**5 variables cauchy
N = 10**5
tiempo_cauchy_razon = tiempo_ejecucion(lambda: estimar_esperanza(N, generar_cauchy))[0]
tiempo_cauchy_TI    = tiempo_ejecucion(lambda: estimar_esperanza(N, generar_cauchy_TI))[0]

print(f"""
Tiempo necesario para estimar {10**5} variables aleatorias
Tiempo razon uniformes      = {tiempo_cauchy_razon} ms
Tiempo transformada inversa = {tiempo_cauchy_TI} ms
""")


Tiempo necesario para estimar 100000 variables aleatorias
Tiempo razon uniformes      = 48.0638 ms
Tiempo transformada inversa = 25.8522 ms



### Ejercicio 13

In [28]:
def eventos_poisson(Lambda, T):
    """
    Genera la variable aleatoria N(T) ligada a un proceso de poisson homogeneo
    de parametro Lambda usando el hecho de que el tiempo entre arribos es una 
    variable exponencial de parametro Lambda
    Args:
        Lambda: Intensidad del proceso de poisson
        T     : Limite del intervalo [0, T]
    Return:
        N_T    : Cantidad de eventos ocurridos en el intervalo [0, T] 
        eventos: Momento en que ocurrio cada evento 
    """
    t, N_T, eventos = 0, 0, []
    while t < T:
        U  = 1 - random()
        t += - log(U) / Lambda # Exponencial de parametro lambda
        if t <= T:
            N_T += 1
            eventos.append(t)
    return N_T, eventos


def poisson_TI_optimizado(Lambda):
    """
    Genera una variable aleatoria de Poisson de parametro Lambda usando
    el metodo de la transformada inversa con una pequeña optimizacion
    Args:
        Lambda: Parametro de la poisson a generar
    """
    p = e ** -Lambda 
    F = p
    for j in range(1, int(Lambda) + 1):
        p *= Lambda / j
        F += p
    U = random()
    if U >= F:
        j = int(Lambda) + 1
        while U >= F:
            p *= Lambda / j
            F += p
            j += 1
        return j-1
    else:
        j = int(Lambda)
        while U < F:
            F -= p
            p *= j / Lambda
            j -= 1
        return j + 1
    
def eventos_poisson_alt(Lambda, T):
    """
    Genera la variable aleatoria N(T) ligada a un proceso de poisson homogeneo
    de parametro Lambda usando el hecho de que dado N(T), la cual es una poisson,
    el tiempo de arribos del proceso involucrado es uniforme en [0, T]
    Args:
        Lambda: Intensidad del proceso de poisson
        T     : Limite del intervalo [0, T]
    Return:
        N_T    : Cantidad de eventos ocurridos en el intervalo [0, T] 
        eventos: Momento en que ocurrio cada evento 
    """
    N_T     = poisson_TI_optimizado(Lambda * T)
    eventos = []
    for _ in range(N_T):
        eventos.append(random()*T)
    eventos.sort()
    return N_T, eventos

### Ejercicio 14

In [29]:
def simular_aficionados(Lambda, t):
    """
    Simula la situacion descrita en el ejercicio 14.
    Args:
        Lambda: Intensidad del proceso de Poisson
        t     : Duracion del periodo que simularemos
    return:
        Cantidad de fanaticos que llegaron al evento en un periodo de tiempo t
    """
    n_colectivos = poisson_TI_optimizado(Lambda * t)
    n_fanaticos  = 0
    for _ in range(n_colectivos):
        n_fanaticos += int(random()*21) + 20
    return n_fanaticos

### Ejercicio 15

In [37]:
def proceso_poisson_no_homo(Lambda, f_Lambda, T):
    """
    Genera un proceso de poisson no homogeneo con intensidad f_Lambda
    a partir de un proceso de poisson homogeneo
    Args:
        Lambda  : Intensidad del proceso homogeneo
        f_Lambda: Funcion de intensidad del proceso no homogeneo
        T       : Longitud del intervalo [0, T]
    Return:
        Nt      : Cantidad total de eventos ocurridos en tiempo T
        eventos : Tiempo de cada uno de los eventos
    """
    NT      = 0
    eventos = []
    t = - log(1 - random()) / Lambda
    while t <= T:
        V = random()
        if V <= f_Lambda(t) / Lambda:
            NT += 1
            eventos.append(t)
        t += -log(1 - random()) / Lambda
    return NT, eventos

def proceso_poisson_no_homo_opt(Lambdas, f_Lambdas, intervalos, T):
    """
    Genera un proceso de poisson no homogeneo con intensidad f_Lambda
    a partir de un proceso de poisson homogeneo (un proceso distinto por
    cada intervalo)
    Args:
        Lambdas    : Lista de intensidades del proceso homogeneo
        f_Lambdas  : Lista de funciones de intensidad del proceso no homogeneo
        intervalos : Lista de intervalos en el cual partimos el dominio 
                     de la funcion de intensidad.
        T          : Longitud del intervalo [0, T]
    Return:
        Nt      : Cantidad total de eventos ocurridos en tiempo T
        eventos : Tiempo de cada uno de los eventos
    """
    j  = 0   # Lo usamos para recorrer los intervalos
    t  = -log(1-random()) / Lambdas[j]
    NT = 0
    eventos = []
    while t <= T:
        if t <= intervalos[j]:
            v = random()
            if v <= f_Lambdas[j](t) / Lambdas[j]: # Decidimos si contar el evento
                NT += 1
                eventos.append(t)
            t += -log(1-random()) / Lambdas[j]
        else: # El ultimo evento quedo fuera del rango del proceso j 
            # Convertimos el excedente, que es una variable exponencial, en una variable
            # exponencial del parametro lambda apropiado
            t = intervalos[j] + (t - intervalos[j]) * Lambdas[j] / Lambdas[j+1]
            j += 1
    return NT, eventos

In [40]:
# Ejercicio a
# Parametros para el inciso i
f_Lambda_i = lambda t: (3 + 4/(t+1)) * (0<=t<=3)
Lambda_i   = 7

# Parametros para el inciso ii
f_Lambda_ii = lambda t: ((t-2)**2 - 5*t + 17) * (0<=t<=5) 
Lambda_ii   = 21

# Parametros para el inciso iii
f_Lambda_iii = lambda t: (t/2 + 1) * (2<=t<=3) + (1 - t/6) * (3<=t<=6)
Lambda_iii   = 0.5

# Ejercicio b
# Parametros para el inciso i
Lambdas_i    = [7, 5, 13/3]
f_Lambdas_i  = [f_Lambda_ii] * 3
intervalos_i = [1, 2, 3]

# Parametros para el inciso ii
Lambdas_ii    = [21, 7, 11]
f_Lambdas_ii  = [f_Lambda_ii] * 3
intervalos_ii = [2, 4, 5]

# Parametros para el inciso iii
Lambdas_iii    = [0.5, 0.5, 1/3]
f_Lambdas_iii  = [f_Lambda_iii] * 3
intervalos_iii = [3, 4, 6]